Feedforward connection after conv2 before deconv3

https://arxiv.org/ftp/arxiv/papers/1708/1708.09126.pdf

In [1]:
import os
import scipy.misc as scm
import numpy as np

def make_project_dir(project_dir):
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
        os.makedirs(os.path.join(project_dir, 'models'))
        os.makedirs(os.path.join(project_dir, 'result'))
        os.makedirs(os.path.join(project_dir, 'result_test'))


def get_image(img_path): # [0,255] to [-1,1]
    img = scm.imread(img_path) * 2. /255. - 1.
    img = img[..., ::-1]  # rgb to bgr
    return img

def get_label(path, size):
    label = int(path[-5])
    one_hot = np.zeros(size)
    one_hot[ label ] = 0.9
    one_hot[ one_hot==0 ] = -0.9
    return one_hot

def inverse_image(img): # [-1,1] to [0,255]
    img = (img + 1.) / 2. * 255.
    img[img > 255] = 255
    img[img < 0] = 0
    img = img[..., ::-1] # bgr to rgb
    return img

def pair_expressions(paths):
    subject_exprs = []
    subject_pairs = []
    all_pairs = []
    last_subject = 0

    # Pair all expression of a subject
    for path in paths:
        subject = int(path[-9:-6])

        if subject != last_subject and last_subject != 0:
            subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
            all_pairs.extend(subject_pairs)
            subject_exprs = []

        subject_exprs.append(path)
        last_subject = subject

    # Last subject
    subject_pairs = [(x, y) for x in subject_exprs for y in subject_exprs]
    all_pairs.extend(subject_pairs)
    return all_pairs

def get_shape_c(tensor): # static shape
    return tensor.get_shape().as_list()


In [2]:
import tensorflow as tf
import numpy as np

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits

def conv(x, filter_shape, bias=True, stride=1, padding="VALID", name="conv2d"):
    kw, kh, nin, nout = filter_shape

    stddev = np.sqrt(2.0/(np.sqrt(nin*nout)*kw*kh))
    k_initializer = tf.truncated_normal_initializer(stddev=stddev)
    
    with tf.variable_scope(name):
        x = tf.layers.conv2d(x, filters=nout, kernel_size=(kw, kh), strides=(stride, stride), padding=padding, 
                             use_bias=bias, kernel_initializer=k_initializer)
    return x

def deconv(x, filter_shape, bias=True, stride=1, padding="VALID", name="conv2d_transpose"):
    kw, kh, nin, nout = filter_shape

    stddev = np.sqrt(1.0/(np.sqrt(nin*nout)*kw*kh))
    k_initializer = tf.random_normal_initializer(stddev=stddev)
    with tf.variable_scope(name):
        x = tf.layers.conv2d_transpose(x, filters=nout, kernel_size=(kw, kh), strides=(stride, stride), padding=padding, 
                                       use_bias=bias, kernel_initializer=k_initializer)
    return x

def fc(x, output_shape, bias=True, name='fc'):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    stddev = np.sqrt(1.0/(np.sqrt(input_shape*output_shape)))
    initializer = tf.random_normal_initializer(stddev=stddev)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        if bias:
            b = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def pool(x, r=2, s=1):
    return tf.nn.avg_pool(x, ksize=[1, r, r, 1], strides=[1, s, s, 1], padding="SAME")


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))

def l2_loss(x, y):
    return tf.reduce_mean(tf.square(x - y))

def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

In [3]:
import glob
import time
import numpy as np
import tensorflow as tf

class op_base:
    def __init__(self, sess, project_name):
        self.sess = sess

        # Train
        self.flag = True #args.flag
        self.gpu_number = 0 #args.gpu_number
        self.project = project_name #"test_began" #args.project

        # Train Data
        self.data_dir = "./Face_data/Faces_with_expression_label/dataset_64x64" #args.data_dir #./Data
        self.dataset = "expr" #args.dataset  # celeba
        self.data_size = 64 #args.data_size  # 64 or 128
        self.data_opt = "crop" #args.data_opt  # raw or crop
        self.data_label_vector_size = 7 #size of one-hot-encoded label vector

        # Train Iteration
        self.niter = 5000 #50 #args.niter
        self.niter_snapshot = 500 #args.nsnapshot
        self.max_to_keep = 5 #args.max_to_keep

        # Train Parameter
        self.batch_size = 32 #16 #args.batch_size
        self.learning_rate = 1e-4 #args.learning_rate
        self.mm = 0.5 #args.momentum
        self.mm2 = 0.999 #args.momentum2
        self.lamda = 0.001 #args.lamda
        self.gamma = 0.5 #args.gamma
        self.filter_number = 64 #args.filter_number
        self.input_size = 64 #args.input_size
        self.embedding = 128 #64 #args.embedding
        self.alpha_1 = 100.0 # AE_L2_loss
        self.alpha_2 = 1.0 # AE_D_z_loss
        self.alpha_3 = 1.0 # AE_D_img_loss
        

        # Result Dir & File
        self.project_dir = 'assets_ae/{0}_{1}_{2}_{3}/'.format(self.project, self.dataset, self.data_opt, self.data_size)
        self.ckpt_dir = os.path.join(self.project_dir, 'models')
        self.model_name = "{0}.model".format(self.project)
        self.ckpt_model_name = os.path.join(self.ckpt_dir, self.model_name)

        # etc.
        if not os.path.exists('assets_ae'):
            os.makedirs('assets_ae')
        make_project_dir(self.project_dir)

    def load(self, sess, saver, ckpt_dir):
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(ckpt_dir, ckpt_name))

In [4]:
import glob
import time
import datetime


class Operator(op_base):
    def __init__(self, sess, project_name):
        op_base.__init__(self, sess, project_name)
        self.build_model()

    def build_model(self):
        # Input placeholder
        self.x = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='x')
        self.y = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='y')
        self.c = tf.placeholder(tf.float32, shape=[None, self.data_label_vector_size], name='c')
        self.pz = tf.placeholder(tf.float32, shape=[None, self.embedding], name='z_prior')
        self.lr = tf.placeholder(tf.float32, name='lr')

        # AE
        self.e1, self.z = self.encoder(self.x)
        self.ae = self.decoder(self.e1, self.z, self.c)
        
        self.e1_test, self.z_test = self.encoder(self.x, reuse=True)
        self.ae_test = self.decoder(self.e1_test, self.z_test, self.c, reuse=True)
        
        # Discriminators
        self.D_z, self.D_z_logits = self.discriminator_z(self.z)
        self.D_pz, self.D_pz_logits = self.discriminator_z(self.pz, reuse=True)
        
        self.D_ae, self.D_ae_logits = self.discriminator_img(self.ae, self.c)
        self.D_real, self.D_real_logits = self.discriminator_img(self.y, self.c, reuse=True)

        # Loss in AE
        self.ae_l2_loss = l2_loss(self.y, self.ae)
        # Loss in D_z
        self.d_z_loss = self.alpha_2 * (
                        tf.reduce_mean(cross_entropy(logits=self.D_pz_logits, labels=tf.ones_like(self.D_pz_logits))) 
                        + tf.reduce_mean(cross_entropy(logits=self.D_z_logits, labels=tf.zeros_like(self.D_z_logits))))
        self.ae_d_z_loss = tf.reduce_mean(cross_entropy(logits=self.D_z_logits, labels=tf.ones_like(self.D_z_logits)))
        # Loss in D_img
        self.d_img_loss = self.alpha_3 * (
                          tf.reduce_mean(cross_entropy(logits=self.D_real_logits, labels=tf.ones_like(self.D_real_logits)))
                          + tf.reduce_mean(cross_entropy(logits=self.D_ae_logits, labels=tf.zeros_like(self.D_ae_logits))))
        self.ae_d_img_loss = tf.reduce_mean(cross_entropy(logits=self.D_ae_logits, labels=tf.ones_like(self.D_ae_logits)))
        # Total loss for AE
        self.ae_loss = self.alpha_1 * self.ae_l2_loss + self.alpha_2 * self.ae_d_z_loss + self.alpha_3 * self.ae_d_img_loss

        # Variables
        ae_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "ae_")
        d_z_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "discriminator_z")
        d_img_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "discriminator_img")

        # Optimizer
        self.opt_ae = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.ae_loss, var_list=ae_vars)
        self.opt_d_z = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.d_z_loss, var_list=d_z_vars)
        self.opt_d_img = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.d_img_loss, var_list=d_img_vars)

        # initializer
        self.sess.run(tf.global_variables_initializer())

        # tf saver
        self.saver = tf.train.Saver(max_to_keep=(self.max_to_keep))

        try:
            self.load(self.sess, self.saver, self.ckpt_dir)
        except:
            # save full graph
            self.saver.save(self.sess, self.ckpt_model_name, write_meta_graph=True)

        # Summary
        if self.flag:
            tf.summary.scalar('loss/ae_loss', self.ae_loss)
            tf.summary.scalar('loss/ae_l2_loss', self.ae_l2_loss)
            tf.summary.scalar('loss/ae_d_z_loss', self.ae_d_z_loss)
            tf.summary.scalar('loss/ae_d_img_loss', self.ae_d_img_loss)
            tf.summary.scalar('loss/d_z_loss', self.d_z_loss)
            tf.summary.scalar('loss/d_img_loss', self.d_img_loss)
            self.merged = tf.summary.merge_all()
            self.writer = tf.summary.FileWriter(self.project_dir, self.sess.graph)

    def train(self, train_flag):
        # load data
        data_path = self.data_dir
        #test_data_path = "./Face_data/Faces_with_expression_label/dataset_64x64" # expressions

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)
            
        print('Shuffle ....')
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        print('Shuffle Done')

        # initial parameter
        start_time = time.time()
        lr = np.float32(self.learning_rate)
        self.count = 0

        for epoch in range(self.niter):
            batch_idxs = len(data) // self.batch_size
            
            for idx in range(0, batch_idxs):
                self.count += 1

                batch_files = data[idx * self.batch_size: (idx + 1) * self.batch_size]
                batch_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
                batch_targets = [get_image(batch_file[1]) for batch_file in batch_files]
                batch_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]
#                 batch_z_prior = np.random.uniform(low=-1.0, high=1.0, 
#                                                  size=[self.batch_size, self.embedding]).astype(np.float32)
                batch_z_prior = np.random.normal(size=[self.batch_size, self.embedding]).astype(np.float32)
                
                # opt & feed list 
                fetches = [self.opt_ae, self.opt_d_z, self.opt_d_img, 
                           self.ae_loss, self.ae_l2_loss, self.ae_d_z_loss, self.ae_d_img_loss,
                           self.d_z_loss, self.d_img_loss, self.merged]
                feed_dict = {self.x: batch_inputs, self.y: batch_targets, self.c: batch_target_labels, 
                             self.pz: batch_z_prior, self.lr: lr}

                # run tensorflow
                _, _, _, loss_ae, loss_ae_l2, loss_ae_d_z, loss_ae_d_img, loss_d_z, loss_d_img, summary = self.sess.run(
                    fetches, feed_dict=feed_dict)
                  
                if self.count % 100 == 1:
                    print("Epoch: [%2d] [%4d/%4d] time: %4.4f, "
                          "loss_ae: %.6f, loss_ae_l2: %.6f, loss_ae_d_z: %.6f, loss_ae_d_img: %.6f, "
                          "loss_d_z: %.6f, loss_d_img: %.6f"
                          % (epoch, idx, batch_idxs, time.time() - start_time,
                             loss_ae, loss_ae_l2, loss_ae_d_z, loss_ae_d_img, loss_d_z, loss_d_img))

                # write train summary
                self.writer.add_summary(summary, self.count)

                # Test during Training
                if self.count % self.niter_snapshot == (self.niter_snapshot - 1):
                    # save & test
                    self.saver.save(self.sess, self.ckpt_model_name, global_step=self.count, write_meta_graph=False)
                    self.test_expr(train_flag)
                    self.test_celebra(train_flag)

    def test_celebra(self, train_flag=True):
        print('Test Sample Generation...')
        # generate output
        img_num = 8*8
        output_f = int(np.sqrt(img_num))
        in_img_num = output_f
        img_size = self.data_size
        gen_img_num = img_num - output_f
        label_size = self.data_label_vector_size
        
        # load data test
        data_path = "./Face_data/Celeba/dataset_64x64" # expressions

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)

        # shuffle test data
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]

        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])

        test_files = data[0: output_f]
        test_data = [get_image(test_file) for test_file in test_files]
        test_data = np.repeat(test_data, [label_size]*in_img_num, axis=0)
        test_data_o = [scm.imread(test_file) for test_file in test_files]
        
        # get one-hot labels
        int_labels = list(range(label_size))
        one_hot = np.zeros((label_size, label_size))
        one_hot[np.arange(label_size), int_labels] = 1
        target_labels = np.tile(one_hot, (output_f, 1))
        
        
        output_gen = (self.sess.run(self.ae_test, feed_dict={self.x: test_data, 
                                                             self.c: target_labels}))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(gen_img_num)]

        for i in range(output_f):
            for j in range(output_f):
                if j == 0:
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_data_o[i]
                else:
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[(j-1) + (i * int(output_f-1))]

        # output save
        scm.imsave(self.project_dir + '/result/' + str(self.count) + '_celebra_output.bmp', im_output_gen)
        
    def test_expr(self, train_flag=True):
        print('Train Sample Generation...')
        # generate output
        img_num =  36 #self.batch_size
        display_img_num = int(img_num / 3)
        img_size = self.data_size

        output_f = int(np.sqrt(img_num))
        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])
        
        # load data
        data_path = self.data_dir

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            data = pair_expressions(data)
            np.save(data_path + '.npy', data)

        # Test data shuffle
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        
        batch_files = data[0: display_img_num]
        test_inputs = [get_image(batch_file[0]) for batch_file in batch_files]
        test_inputs_o = [scm.imread((batch_file[0])) for batch_file in batch_files]
        test_targets = [scm.imread((batch_file[1])) for batch_file in batch_files]
        test_target_labels = [get_label(batch_file[1], self.data_label_vector_size) for batch_file in batch_files]

        output_gen = (self.sess.run(self.ae_test, feed_dict={self.x: test_inputs, 
                                                             self.c: test_target_labels}))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(display_img_num)]

        for i in range(output_f): # row
            for j in range(output_f): # col
                if j % 3 == 0: # input img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_inputs_o[int(j / 3) + (i * int(output_f / 3))]
                elif j % 3 == 1: # output img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[int(j / 3) + (i * int(output_f / 3))]
                else: # target img
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_targets[int(j / 3) + (i * int(output_f / 3))]
                   

        labels = np.argmax(test_target_labels, axis=1)
        label_string = ''.join(str(int(l)) for l in labels)
        # output save
        scm.imsave(self.project_dir + '/result/' + str(self.count) + '_' + label_string 
                   + '_expr_output.bmp', im_output_gen)
        

In [5]:

class AE(Operator):
    def __init__(self, sess, project_name):
        Operator.__init__(self, sess, project_name)


    def encoder(self, x, reuse=None):
        with tf.variable_scope('ae_encoder') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"

            x = conv(x, [5, 5, 3, f], stride=2, padding=p, name='conv1_a')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f, f], stride=1, padding=p, name='conv1_b')
            x = tf.nn.elu(x)

            x = conv(x, [5, 5, f, f*2], stride=2, padding=p, name='conv2_a')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*2, f*2], stride=1, padding=p, name='conv2_b')
            x = tf.nn.elu(x)
            
            x_1 = x # feed forward
            
            x = conv(x, [5, 5, f*2, f*4], stride=2, padding=p, name='conv3_a')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*4, f*4], stride=1, padding=p, name='conv3_b')
            x = tf.nn.elu(x)
            
            x = conv(x, [5, 5, f*4, f*4], stride=1, padding=p, name='conv4_a')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*4, f*4], stride=1, padding=p, name='conv4_b')
            x = tf.nn.elu(x)

            x = conv(x, [5, 5, f*4, f*8], stride=2, padding=p, name='conv5_a')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*8, f*8], stride=1, padding=p, name='conv5_b')
            x = tf.nn.elu(x)
            
            x = tf.reshape(x, [-1, 1, 1, 4*4*f*8])
            
            x = fc(x, h, name='fc')
            x = tf.nn.tanh(x)
            
        return x_1, x

    def decoder(self, x_1, x, c, reuse=None):
        with tf.variable_scope('ae_decoder') as scope:
            if reuse:
                scope.reuse_variables()

            w = self.data_size
            f = self.filter_number
            p = "SAME"
            
            x = tf.concat([x, c], axis=1) # adding label information

            x = fc(x, 4*4*f*8, name='fc')
            x = tf.reshape(x, [-1, 4, 4, f*8])

            x = deconv(x, [5, 5, f*8, f*4], stride=2, padding=p, name='deconv1')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*4, f*4], stride=1, padding=p, name='conv1')
            x = tf.nn.elu(x)
            
            x = deconv(x, [5, 5, f*4, f*2], stride=2, padding=p, name='deconv2')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*2, f*2], stride=1, padding=p, name='conv2')
            x = tf.nn.elu(x)
            
            x = deconv(x, [5, 5, f*2, f*2], stride=1, padding=p, name='deconv3')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*2, f*2], stride=1, padding=p, name='conv3')
            x = tf.nn.elu(x)
            
            x = x + x_1
            
            x = deconv(x, [5, 5, f*2, f*2], stride=2, padding=p, name='deconv4')
            x = tf.nn.elu(x)
            x = conv(x, [5, 5, f*2, f], stride=1, padding=p, name='conv4')
            x = tf.nn.elu(x)
            
            x = deconv(x, [5, 5, f, f], stride=2, padding=p, name='deconv5')
            x = conv(x, [5, 5, f, 3], stride=1, padding=p, name='conv5')
            x = tf.nn.elu(x)
            x = tf.nn.tanh(x)
            
        return x
    
    def discriminator_z(self, z, reuse=None):
        with tf.variable_scope('discriminator_z') as scope:
            if reuse:
                scope.reuse_variables()
                
            x = fc(z, 256, name='fc_1')
            x = fc(x, 128, name='fc_2')
            x = fc(x, 64, name='fc_3')
            x = fc(x, 32, name='fc_4')
            x = fc(x, 16, name='fc_5')
            x = fc(x, 1, name='fc_6')
            
        return tf.nn.sigmoid(x), x
    
    def discriminator_img(self, img, c, reuse=None):
        with tf.variable_scope('discriminator_img') as scope:
            if reuse:
                scope.reuse_variables()
            
            f = 16
            c_num = self.data_label_vector_size
            p = "SAME"

            x = conv(img, [5, 5, 3, f], stride=2, padding=p, name='conv1')
            x = tf.nn.elu(x)
            
            # Add classes as channels to the outcome of first layer as described in IcGAN paper
            x = tf.concat([x, tf.tile(tf.reshape(c, [-1, 1, 1, get_shape_c(c)[-1]]),\
                                      [1, x.get_shape().as_list()[1], x.get_shape().as_list()[2], 1])],\
                                      axis=3)

            x = conv(x, [5, 5, f+c_num, f*2], stride=2, padding=p, name='conv2')
            x = tf.nn.elu(x)
            
            x = conv(x, [5, 5, f*2, f*4], stride=2, padding=p, name='conv3')
            x = tf.nn.elu(x)
            
            x = conv(x, [5, 5, f*4, f*8], stride=2, padding=p, name='conv4')
            x = tf.nn.elu(x)
            
            x = tf.reshape(x, [-1, 1, 1, 4*4*f*8])
            
            x = fc(x, 1024, name='fc_1')
            x = fc(x, 1, name='fc_2')
            
        return tf.nn.sigmoid(x), x


In [6]:
import distutils.util
import os
import tensorflow as tf

''' config settings '''

project_name = "CDAAE_Face_9_"
train_flag = True

'''-----------------'''

gpu_number = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #args.gpu_number

with tf.device('/gpu:{0}'.format(gpu_number)):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        model = AE(sess, project_name)

        # TRAIN / TEST
        if train_flag:
            model.train(train_flag)
        else:
            model.test(train_flag)

Shuffle ....
Shuffle Done
Epoch: [ 0] [   0/ 316] time: 1.0566, loss_ae: 38.678905, loss_ae_l2: 0.358783, loss_ae_d_z: 0.548716, loss_ae_d_img: 2.251904, loss_d_z: 1.965354, loss_d_img: 2.244192
Epoch: [ 0] [ 100/ 316] time: 14.0615, loss_ae: 10.125440, loss_ae_l2: 0.051564, loss_ae_d_z: 3.880579, loss_ae_d_img: 1.088420, loss_d_z: 0.849079, loss_d_img: 2.082426
Epoch: [ 0] [ 200/ 316] time: 27.0960, loss_ae: 10.278779, loss_ae_l2: 0.054225, loss_ae_d_z: 4.398956, loss_ae_d_img: 0.457320, loss_d_z: 0.573631, loss_d_img: 1.944749
Epoch: [ 0] [ 300/ 316] time: 40.1665, loss_ae: 10.739998, loss_ae_l2: 0.052203, loss_ae_d_z: 4.759795, loss_ae_d_img: 0.759923, loss_d_z: 0.264183, loss_d_img: 1.401215
Epoch: [ 1] [  84/ 316] time: 53.2691, loss_ae: 11.462908, loss_ae_l2: 0.063019, loss_ae_d_z: 4.656425, loss_ae_d_img: 0.504548, loss_d_z: 0.196512, loss_d_img: 1.583944
Train Sample Generation...
Test Sample Generation...
Epoch: [ 1] [ 184/ 316] time: 67.1193, loss_ae: 11.271939, loss_ae_l2: 0

KeyboardInterrupt: 